In [ ]:
# emotional tts quick c1 iteration

In [ ]:
# downloading the datasets

# !(mkdir /DataEmotionalTTS && \

!(cd /DataEmotionalTTS && \
    wget https://www.openslr.org/resources/110/thorsten-emotional_v02.tgz && \
    tar -zxvf thorsten-emotional_v02.tgz)

In [3]:
# create manifests, normalization and phonemization

# !(cd NeMoEmotionalTTS && \
#     python get_data.py \

!(python scripts/dataset_processing/tts/openslr_emotional/get_data.py \
        --data-root ../DataEmotionalTTS/ \
        --val-size 0.1 \
        --test-size 0.2)

[NeMo W 2022-07-18 17:58:34 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
      warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)
    
[NeMo I 2022-07-18 17:58:35 tokenize_and_classify:81] Creating ClassifyFst grammars. This might take some time...
Created ../DataEmotionalTTS/thorsten-emotional_v02/cache_dir/_cased_de_tn_True_deterministic.far
[NeMo I 2022-07-18 17:58:50 tokenize_and_classify:144] ClassifyFst grammars are saved to ../DataEmotionalTTS/thorsten-emotional_v02/cache_dir/_cased_de_tn_True_deterministic.far.
Created ../DataEmotionalTTS/thorsten-emotional_v02/cache_dir/de_tn_True_deterministic_verbalizer.far
[NeMo I 2022-07-18 17:58:53 verbalize_final:71] VerbalizeFinalFst grammars are saved to ../DataEmotionalTTS/thorsten-emotional_v02/cache_dir/de_tn_True_deterministic_verbalizer.far.


In [ ]:
# 2cc2cc4a34b961ef1657cc82dbd18875 does not exist for whisper category

In [4]:
# phonemization
## inside the phonemizer container

from phonemizer.backend import EspeakBackend
import json

backend = EspeakBackend('de')
unique_symbols = set()
input_manifest_filepaths = ["/DataEmotionalTTS/thorsten-emotional_v02/train_manifest", \
                            "/DataEmotionalTTS/thorsten-emotional_v02/test_manifest", \
                            "/DataEmotionalTTS/thorsten-emotional_v02/val_manifest"]

for input_manifest_filepath in input_manifest_filepaths:
    output_manifest_filepath = input_manifest_filepath+"_phonemes"
    records = []
    n_text = []
    with open(input_manifest_filepath + ".json", "r") as f:
        for i, line in enumerate(f):
            d = json.loads(line)
            records.append(d)
            n_text.append(d['normalized_text'])

    phonemized = backend.phonemize(n_text)
    for line in n_text:
        for char in line:
            unique_symbols.add(char)
    new_records = []
    for i in range(len(records)):
        records[i]["is_phoneme"] = 0
        new_records.append(records[i])
        phoneme_record = records[i].copy()
        phoneme_record["normalized_text"] = phonemized[i]
        phoneme_record["is_phoneme"] = 1
        new_records.append(phoneme_record)

    with open(output_manifest_filepath + ".json", "w") as f:
        for r in new_records:
            f.write(json.dumps(r) + '\n')
print(unique_symbols)

{'O', 't', 'T', 'Ü', '?', '.', 'ö', 'V', 'n', 'w', 'B', 'Z', 'N', 'A', 'm', 'g', 'P', 'f', 'e', 'z', 'D', 'S', 'k', 'j', 'K', 'E', 'h', 'b', 'ü', '-', '!', 'C', 'I', 'W', 'F', 'ä', 'x', 'r', 'U', 'M', 'u', 's', 'G', 'd', 'o', 'c', 'l', 'a', 'i', ' ', ':', 'J', 'L', 'p', 'R', '"', 'y', 'ß', "'", ',', 'v', 'H'}


In [7]:
a = ''.join(unique_symbols)

In [13]:
a = a.lower()

In [14]:
[char for char in a if char not in "ʊʃŋɜːɛɾəɪçɔøɡœɑÜ„1Q̃ɒʒÄɹÖʌθàó̈ðéɐáabcdefghijklmnopqrstuvwxyzäöüß"]

['?', '.', '-', '!', ' ', ':', '"', "'", ',']

In [6]:
!ls /DataEmotionalTTS/thorsten-emotional_v02

README.md  neutral			    train_manifest.json
amused	   sleepy			    train_manifest_phonemes.json
angry	   surprised			    val_manifest.json
cache_dir  test_manifest.json		    val_manifest_phonemes.json
disgusted  test_manifest_phonemes.json	    whisper
drunk	   thorsten-emotional-metadata.csv


In [15]:
# dataset config
!cat scripts/dataset_processing/tts/openslr_emotional/ds_conf/ds_for_fastpitch_align.yaml

name: "ds_for_fastpitch_align"

manifest_filepath: "train_manifest.json"
sup_data_path: "sup_data"
sup_data_types: [ "align_prior_matrix", "pitch" ]
whitelist_path: "nemo_text_processing/text_normalization/de/data/whitelist.tsv"

dataset:
  _target_: nemo.collections.tts.torch.data.TTSDataset
  manifest_filepath: ${manifest_filepath}
  sample_rate: 22050
  sup_data_path: ${sup_data_path}
  sup_data_types: ${sup_data_types}
  n_fft: 1024
  win_length: 1024
  hop_length: 256
  window: "hann"
  n_mels: 80
  lowfreq: 0
  highfreq: null
  max_duration: null
  min_duration: 0.1
  ignore_file: null
  trim: false
  pitch_fmin: 65.40639132514966
  pitch_fmax: 2093.004522404789

  text_normalizer:
    _target_: nemo_text_processing.text_normalization.normalize.Normalizer
    lang: de
    input_case: cased
    whitelist: ${whitelist_path}

  text_normalizer_call_kwargs:
    verbose: false
    punct_pre_process: true
    punct_post_process: true

  text_tokenizer:
    _target_: nemo.collections.tt

In [2]:
!which python

/opt/conda/bin/python


In [4]:
# supplementary data creation (optional)
!(PYTHONPATH=. python scripts/dataset_processing/tts/extract_sup_data.py \
        --config-path /NeMoEmotionalTTS/scripts/dataset_processing/tts/openslr_emotional/ds_conf \
        --config-name ds_for_fastpitch_align.yaml \
        manifest_filepath=/DataEmotionalTTS/thorsten-emotional_v02/train_manifest_phonemes.json \
        sup_data_path=/DataEmotionalTTS/thorsten-emotional_v02/phonemes/)

/opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)
[NeMo W 2022-07-27 01:56:39 experimental:27] Module <class 'nemo.collections.tts.torch.tts_tokenizers.IPATokenizer'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo I 2022-07-27 01:56:40 tokenize_and_classify:81] Creating ClassifyFst grammars. This might take some time...
Error executing job with overrides: ['manifest_filepath=/DataEmotionalTTS/thorsten-emotional_v02/train_manifest_phonemes.json', 'sup_data_path=/DataEmotionalTTS/thorsten-emotional_v02/phonemes/']
Error in call to target 'nemo.collections.tts.torch.data.TTSDataset':
FileNotFoundError(2, 'No such file or directory')
full_key: dataset

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


In [ ]:
# download the pretrained model, run on host
# ngc registry model download-version "nvstaging/nemo/tts_de_fastpitchhifigan:1.10.0"

In [ ]:
# pre-train fastpitch (with speaker and emotional embedding) on German neutral tts
!(CUDA_VISIBLE_DEVICES=0 python examples/tts/fastpitch.py --config-path conf/de --config-name fastpitch_align_22050 \
    name="c1e1-FastPitch-pretrain" \
    model.train_ds.dataloader_params.batch_size=32 \
    model.validation_ds.dataloader_params.batch_size=32 \
    train_dataset=/DataEmotionalTTS/openslr-95-german-neutral-tts/thorsten-de/train_manifest_phonemes.json \
    validation_datasets=/DataEmotionalTTS/openslr-95-german-neutral-tts/thorsten-de/val_manifest_phonemes.json \
    sup_data_path=/DataEmotionalTTS/openslr-95-german-neutral-tts/thorsten-de/phonemes5/ \
    whitelist_path=nemo_text_processing/text_normalization/de/data/whitelist.tsv \
    exp_manager.exp_dir=/resultGermanTTS \
    trainer.max_epochs=100 \
    pitch_mean=132.524658203125 \
    pitch_std=pitch_std: 37.389366149902344 \
    +exp_manager.create_wandb_logger=true \
    +exp_manager.wandb_logger_kwargs.name="072620221311_c1e1-FastPitch-pretrain" \
    +exp_manager.wandb_logger_kwargs.project="EmotionalTTS" | tee -a 072620221311_c1e1-FastPitch-pretrain.log)

In [ ]:
# monitor performance of pre-trained fastpitch


In [ ]:
# finetune the model trained in the last cell
!(CUDA_VISIBLE_DEVICES=0 python examples/tts/fastpitch_finetune.py --config-path conf/de --config-name fastpitch_align_v1.05.yaml \
    name="c1e1-FastPitch-finetune" \
    model.train_ds.dataloader_params.batch_size=32 \
    model.validation_ds.dataloader_params.batch_size=32 \
    train_dataset=/DataEmotionalTTS/thorsten-emotional_v02/train_manifest_phonemes.json \
    validation_datasets=/DataEmotionalTTS/thorsten-emotional_v02/val_manifest_phonemes.json \
    sup_data_path=/DataEmotionalTTS/thorsten-emotional_v02/train_manifest_phonemes.json \
    whitelist_path=nemo_text_processing/text_normalization/de/data/whitelist.tsv \
    exp_manager.exp_dir=/resultEmotionalTTS \
    +init_from_nemo_model=/DataEmotionalTTS/tts_de_fastpitchhifigan_v1.10.0/tts_de_fastpitch_align.nemo \
    +trainer.max_steps=1000 ~trainer.max_epochs \
    trainer.check_val_every_n_epoch=25 \
    model.n_speakers=1 \
    model.n_emotions=8 \
    model.optim.lr=2e-4 \
    ~model.optim.sched model.optim.name=adamw trainer.devices=1 trainer.strategy=null \
    pitch_mean=169.59390258789062 \
    pitch_std=103.64842224121094 \
    +exp_manager.create_wandb_logger=true \
    +exp_manager.wandb_logger_kwargs.name="072620221311_c1e1-FastPitch-finetune" \
    +exp_manager.wandb_logger_kwargs.project="EmotionalTTS" | tee -a 072620221311_c1e1-FastPitch-pretrain.log)